## Reshaping con objetos de DataFrame

In [4]:
import numpy as np
import pandas as pd
import pandas._testing as tm

In [5]:
def unpivot(frame):
    N, K = frame.shape
    data = {
        "value": frame.to_numpy().ravel("F"),
        "variable": np.asarray(frame.columns).repeat(N),
        "date": np.tile(np.asarray(frame.index), K),
    }
    return pd.DataFrame(data, columns=["date", "variable", "value"])


In [6]:
df = unpivot(tm.makeTimeDataFrame(3))

In [7]:
df

,date,variable,value
0,2000-01-03,A,0.303460
1,2000-01-04,A,0.223136
2,2000-01-05,A,0.536628
3,2000-01-03,B,-0.964065
4,2000-01-04,B,0.598610
5,2000-01-05,B,1.082319
6,2000-01-03,C,0.983640
7,2000-01-04,C,-0.896869
8,2000-01-05,C,0.148936
9,2000-01-03,D,0.232305


In [8]:
# Para seleccionar todo para la variable A, podríamos hacer
filtered = df[df["variable"] == "A"]

In [9]:
filtered

,date,variable,value
0,2000-01-03,A,0.303460
1,2000-01-04,A,0.223136
2,2000-01-05,A,0.536628


### DataFrame.pivot()

#### Para remodelar los datos en esta forma, usamos el método DataFrame.pivot() (también implementado como una función de nivel superior pivot()

In [12]:
# Cuando transformamos un data frame a tabla dínamica debemos definir, de donde proviene la información
# index
# columns
# values

pivoted = df.pivot(index="date", columns="variable", values="value")

In [11]:
pivoted

variable,A,B,C,D
date,,,,
2000-01-03,0.303460,-0.964065,0.983640,0.232305
2000-01-04,0.223136,0.598610,-0.896869,-0.026508
2000-01-05,0.536628,1.082319,0.148936,-0.945854


In [13]:
# Si no definen los valores, automáticamente será creada una tabla jerárquica
df["value2"] = df["value"] * 2

In [14]:
pivoted = df.pivot(index="date", columns="variable")

In [15]:
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03  0.303460 -0.964065  0.983640  0.232305  0.606920 -1.928130   
2000-01-04  0.223136  0.598610 -0.896869 -0.026508  0.446272  1.197219   
2000-01-05  0.536628  1.082319  0.148936 -0.945854  1.073256  2.164638   

                                
variable           C         D  
date                            
2000-01-03  1.967281  0.464610  
2000-01-04 -1.793739 -0.053016  
2000-01-05  0.297872 -1.891707

In [16]:
# Después de aplicar pivot a la tabla, podremos hacer selecciones como de costumbre
pivoted["value2"]

variable,A,B,C,D
date,,,,
2000-01-03,0.606920,-1.928130,1.967281,0.464610
2000-01-04,0.446272,1.197219,-1.793739,-0.053016
2000-01-05,1.073256,2.164638,0.297872,-1.891707


## Reshaping apilando y desapilando (by stacking and unstacking) - stack() and unstack() 

#### Estrechamente relacionados con el método pivot() están los métodos relacionados stack() y unstack() disponibles en Series y DataFrame. Estos métodos están diseñados para funcionar junto con objetos MultiIndex.

#### stack(): "pivot" un nivel de las etiquetas de columna (posiblemente jerárquicas), devolviendo un DataFrame con un índice con un nuevo nivel más interno de etiquetas de fila.

#### unstack(): (operación inversa de stack()) "pivot" un nivel del índice de fila (posiblemente jerárquico) al eje de la columna, produciendo un DataFrame remodelado con un nuevo nivel más interno de etiquetas de columna

In [17]:
tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)

In [18]:
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])

In [19]:
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])

In [22]:
df

A         B
first second                    
bar   one    -0.852334 -1.263781
      two    -0.015823  0.192393
baz   one     0.870592  1.213986
      two     0.320067  1.052152
foo   one     1.137447  1.382694
      two     1.316248 -0.235494
qux   one     0.224005 -0.126731
      two     1.031316 -0.514244

In [20]:
df2 = df[:4]

In [21]:
df2

A         B
first second                    
bar   one    -0.852334 -1.263781
      two    -0.015823  0.192393
baz   one     0.870592  1.213986
      two     0.320067  1.052152

In [23]:
# La función stack() "comprime" un nivel en las columnas de DataFrame
# Si las columnas tienen un índice múltiple, puede elegir qué nivel apilar. 
# El nivel apilado se convierte en el nuevo nivel más bajo en un índice múltiple en las columnas
stacked = df2.stack()

In [24]:
stacked

first  second   
bar    one     A   -0.852334
               B   -1.263781
       two     A   -0.015823
               B    0.192393
baz    one     A    0.870592
               B    1.213986
       two     A    0.320067
               B    1.052152
dtype: float64

In [25]:
# Para descomprimir o volver al estado original utilizamos unstack()
stacked.unstack()

A         B
first second                    
bar   one    -0.852334 -1.263781
      two    -0.015823  0.192393
baz   one     0.870592  1.213986
      two     0.320067  1.052152

In [26]:
stacked.unstack(1)

second        one       two
first                      
bar   A -0.852334 -0.015823
      B -1.263781  0.192393
baz   A  0.870592  0.320067
      B  1.213986  1.052152

In [27]:
stacked.unstack(0)

first          bar       baz
second                      
one    A -0.852334  0.870592
       B -1.263781  1.213986
two    A -0.015823  0.320067
       B  0.192393  1.052152

In [28]:
# Si los indices tienen nombres, podemos utilizarlos para aplicar la funcion
stacked.unstack("second")

second        one       two
first                      
bar   A -0.852334 -0.015823
      B -1.263781  0.192393
baz   A  0.870592  0.320067
      B  1.213986  1.052152

## Apilar y desapilar con Múltiples Niveles

#### También puede apilar o desapilar más de un nivel a la vez pasando una lista de niveles, en cuyo caso el resultado final es como si cada nivel de la lista se procesara individualmente.

In [29]:
columns = pd.MultiIndex.from_tuples(
    [
        ("A", "cat", "long"),
        ("B", "cat", "long"),
        ("A", "dog", "short"),
        ("B", "dog", "short"),
    ],
    names=["exp", "animal", "hair_length"],
)

In [30]:
df = pd.DataFrame(np.random.randn(4, 4), columns=columns)

In [31]:
df

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,0.445209,-0.111500,2.434675,-0.259929
1,1.111994,1.519075,1.113558,-1.558580
2,-1.010088,-0.993280,-0.843242,0.078998
3,0.480633,2.198066,0.019420,0.698096


In [32]:
df.stack(level=["animal", "hair_length"])

exp                          A         B
  animal hair_length                    
0 cat    long         0.445209 -0.111500
  dog    short        2.434675 -0.259929
1 cat    long         1.111994  1.519075
  dog    short        1.113558 -1.558580
2 cat    long        -1.010088 -0.993280
  dog    short       -0.843242  0.078998
3 cat    long         0.480633  2.198066
  dog    short        0.019420  0.698096

In [33]:
# También podemos identificar los niveles por valores numéricos
df.stack(level=[1, 2])

exp                          A         B
  animal hair_length                    
0 cat    long         0.445209 -0.111500
  dog    short        2.434675 -0.259929
1 cat    long         1.111994  1.519075
  dog    short        1.113558 -1.558580
2 cat    long        -1.010088 -0.993280
  dog    short       -0.843242  0.078998
3 cat    long         0.480633  2.198066
  dog    short        0.019420  0.698096

## Reshaping con melt - Convertir valores a columnas

La función **melt()** de nivel superior y el **DataFrame.melt()** correspondiente son útiles para masajear un DataFrame en un formato en el que una o más columnas son variables de identificación, mientras que todas las demás columnas, consideradas variables medidas, están "sin pivotar" en la fila. eje, dejando solo dos columnas sin identificador, "variable" y "valor". Los nombres de esas columnas se pueden personalizar proporcionando los parámetros var_name y value_name

In [34]:
cheese = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    }
)

In [35]:
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [36]:
# Ahora aplicaremos melt(), y convertiremos "first" y "last", en dos nuevas columnas
cheese.melt(id_vars=["first", "last"])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [37]:
cheese.melt(id_vars=["first", "last"], var_name="quantity")

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [41]:
# Veamos ahora un ejemplo, con un multiindice
index = pd.MultiIndex.from_tuples([("person", "A"), ("person", "B")])

In [42]:
cheese = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    },
    index=index,
)

In [43]:
cheese

first last  height  weight
person A  John  Doe     5.5     130
       B  Mary   Bo     6.0     150

In [44]:
cheese.melt(id_vars=["first", "last"])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [45]:
# Para conservar los índices originales, debemos usar el argumento "ignore_index=False"
cheese.melt(id_vars=["first", "last"], ignore_index=False)

first last variable  value
person A  John  Doe   height    5.5
       B  Mary   Bo   height    6.0
       A  John  Doe   weight  130.0
       B  Mary   Bo   weight  150.0

### wide_to_long() similar a melt()

In [46]:
dft = pd.DataFrame(
    {
        "A1970": {0: "a", 1: "b", 2: "c"},
        "A1980": {0: "d", 1: "e", 2: "f"},
        "B1970": {0: 2.5, 1: 1.2, 2: 0.7},
        "B1980": {0: 3.2, 1: 1.3, 2: 0.1},
        "X": dict(zip(range(3), np.random.randn(3))),
    }
)

In [47]:
dft["id"] = dft.index

In [48]:
dft

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,0.195940,0
1,b,e,1.2,1.3,0.588591,1
2,c,f,0.7,0.1,0.150961,2


In [49]:
pd.wide_to_long(dft, ["A", "B"], i="id", j="year")

,,X,A,B
id,year,,,
0,1970,0.195940,a,2.5
1,1970,0.588591,b,1.2
2,1970,0.150961,c,0.7
0,1980,0.195940,d,3.2
1,1980,0.588591,e,1.3
2,1980,0.150961,f,0.1


## Combinando con estadísticas y GroupBy

#### No debería sorprender que la combinación de pivot() / stack() / unstack() con GroupBy y las funciones estadísticas básicas de Series y DataFrame pueda producir algunas manipulaciones de datos muy expresivas y rápidas.

In [54]:
columns = pd.MultiIndex.from_tuples(
    [
        ("A", "cat"),
        ("B", "dog"),
        ("B", "cat"),
        ("A", "dog"),
    ],
    names=["exp", "animal"],
)

In [56]:
index = pd.MultiIndex.from_product(
    [("bar", "baz", "foo", "qux"), ("one", "two")], names=["first", "second"]
)

In [58]:
df = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)

In [59]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.341072 -0.613694 -0.458615  1.400419
      two     2.228768 -0.137789  0.339085 -1.807528
baz   one     0.354889 -0.143549  0.590079  0.701782
      two    -0.286010  0.093535  1.459081 -0.344410
foo   one     0.513846 -1.427431 -0.691387 -1.117198
      two    -0.768869  0.007087 -0.324393 -0.202964
qux   one    -1.826545  0.811314  1.138981  1.897945
      two    -1.208721  0.181832  0.256169  1.643016

In [60]:
# Podemos ver la media de cada columna
df.stack().mean(1).unstack()

animal             cat       dog
first second                    
bar   one    -0.058772  0.393363
      two     1.283926 -0.972658
baz   one     0.472484  0.279116
      two     0.586535 -0.125437
foo   one    -0.088771 -1.272315
      two    -0.546631 -0.097939
qux   one    -0.343782  1.354630
      two    -0.476276  0.912424

In [61]:
df.groupby(level=1, axis=1).mean()

animal             cat       dog
first second                    
bar   one    -0.058772  0.393363
      two     1.283926 -0.972658
baz   one     0.472484  0.279116
      two     0.586535 -0.125437
foo   one    -0.088771 -1.272315
      two    -0.546631 -0.097939
qux   one    -0.343782  1.354630
      two    -0.476276  0.912424

In [62]:
df.stack().groupby(level=1).mean()

exp,A,B
second,,
one,0.283276,-0.099288
two,-0.093340,0.234326


In [63]:
df.mean().unstack(0)

exp,A,B
animal,,
cat,-0.081446,0.288625
dog,0.271383,-0.153587


## Tablas dinámicas - Pivot Tables - pivot_table() 

#### La función pivot_table() se puede usar para crear tablas dinámicas al estilo de una hoja de cálculo. Tiene los siguientes argumentos

#### 1. data: a DataFrame object.
#### 2. values: a column or a list of columns to aggregate.
#### 3. index: a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table index.
#### 4. columns: a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table column.
#### 5. aggfunc: function to use for aggregation, defaulting to numpy.mean.

In [64]:
import datetime

In [65]:
df = pd.DataFrame(
    {
        "A": ["one", "one", "two", "three"] * 6,
        "B": ["A", "B", "C"] * 8,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 4,
        "D": np.random.randn(24),
        "E": np.random.randn(24),
        "F": [datetime.datetime(2013, i, 1) for i in range(1, 13)]
        + [datetime.datetime(2013, i, 15) for i in range(1, 13)],
    }
)

In [67]:
df.head(10)

,A,B,C,D,E,F
0,one,A,foo,0.334915,-1.473241,2013-01-01
1,one,B,foo,-0.899280,-0.485105,2013-02-01
2,two,C,foo,1.199145,-0.765235,2013-03-01
3,three,A,bar,-0.150724,0.569081,2013-04-01
4,one,B,bar,1.655458,-0.273584,2013-05-01
5,one,C,bar,-0.104854,-0.400400,2013-06-01
6,two,A,foo,1.650828,-1.456718,2013-07-01
7,three,B,foo,-1.561268,3.919242,2013-08-01
8,one,C,foo,0.220828,0.588927,2013-09-01
9,one,A,bar,-0.764537,-0.648978,2013-10-01


In [68]:
# Vamos a crear una tabla dinámica
pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"])

C             bar       foo
A     B                    
one   A -0.253236  0.826218
      B  0.769927 -0.848733
      C -0.823565  0.385051
three A  0.346411       NaN
      B       NaN -0.598831
      C  0.872067       NaN
two   A       NaN  0.640759
      B -0.279023       NaN
      C       NaN -0.129406

In [69]:
pd.pivot_table(df, values="D", index=["B"], columns=["A", "C"], aggfunc=np.sum)

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A -0.506472  1.652436  0.692821       NaN       NaN  1.281518
B  1.539855 -1.697466       NaN -1.197661 -0.558045       NaN
C -1.647131  0.770101  1.744135       NaN       NaN -0.258813

In [70]:
pd.pivot_table(
    df, values=["D", "E"],
    index=["B"],
    columns=["A", "C"],
    aggfunc=np.sum,
)

D                                                           E  \
A       one               three                 two                 one   
C       bar       foo       bar       foo       bar       foo       bar   
B                                                                         
A -0.506472  1.652436  0.692821       NaN       NaN  1.281518 -2.223740   
B  1.539855 -1.697466       NaN -1.197661 -0.558045       NaN  1.281528   
C -1.647131  0.770101  1.744135       NaN       NaN -0.258813  1.354066   

                                                     
A               three                 two            
C       foo       bar       foo       bar       foo  
B                                                    
A  0.145691 -0.489828       NaN       NaN  0.204539  
B  1.245934       NaN  4.425121  0.938225       NaN  
C  0.384100 -0.065588       NaN       NaN -0.626744

In [71]:
pd.pivot_table(df[["A", "B", "C", "D", "E"]], index=["A", "B"], columns=["C"])

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A -0.253236  0.826218 -1.111870  0.072846
      B  0.769927 -0.848733  0.640764  0.622967
      C -0.823565  0.385051  0.677033  0.192050
three A  0.346411       NaN -0.244914       NaN
      B       NaN -0.598831       NaN  2.212560
      C  0.872067       NaN -0.032794       NaN
two   A       NaN  0.640759       NaN  0.102270
      B -0.279023       NaN  0.469112       NaN
      C       NaN -0.129406       NaN -0.313372

In [73]:
# Además, puede usar Grouper para palabras clave de índice y columnas
pd.pivot_table(df, values="D", index=pd.Grouper(freq="M", key="F"), columns="C")

C,bar,foo
F,,
2013-01-31,NaN,0.826218
2013-02-28,NaN,-0.848733
2013-03-31,NaN,-0.129406
2013-04-30,0.346411,NaN
2013-05-31,0.769927,NaN
2013-06-30,-0.823565,NaN
2013-07-31,NaN,0.640759
2013-08-31,NaN,-0.598831
2013-09-30,NaN,0.385051


In [74]:
table = df.pivot_table(
    index=["A", "B"],
    columns="C",
    values=["D", "E"],
    margins=True,
    aggfunc=np.std
)

In [75]:
table

D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  0.723088  0.694807  0.850654  0.654628  2.186497  1.484684
      B  1.252330  0.071485  1.182300  1.293083  1.567051  1.173034
      C  1.016412  0.232246  0.921554  1.523720  0.561268  0.978426
three A  0.703055       NaN  0.703055  1.151163       NaN  1.151163
      B       NaN  1.361092  1.361092       NaN  2.413613  2.413613
      C  1.529483       NaN  1.529483  0.332196       NaN  0.332196
two   A       NaN  1.428453  1.428453       NaN  2.204741  2.204741
      B  1.262802       NaN  1.262802  0.757354       NaN  0.757354
      C       NaN  1.878855  1.878855       NaN  0.639030  0.639030
All      1.044737  1.070856  1.013278  1.006862  1.558910  1.273425

In [78]:
table.stack()

D         E
A     B C                      
one   A All  0.850654  1.484684
        bar  0.723088  0.654628
        foo  0.694807  2.186497
      B All  1.182300  1.173034
        bar  1.252330  1.293083
        foo  0.071485  1.567051
      C All  0.921554  0.978426
        bar  1.016412  1.523720
        foo  0.232246  0.561268
three A All  0.703055  1.151163
        bar  0.703055  1.151163
      B All  1.361092  2.413613
        foo  1.361092  2.413613
      C All  1.529483  0.332196
        bar  1.529483  0.332196
two   A All  1.428453  2.204741
        foo  1.428453  2.204741
      B All  1.262802  0.757354
        bar  1.262802  0.757354
      C All  1.878855  0.639030
        foo  1.878855  0.639030
All     All  1.013278  1.273425
        bar  1.044737  1.006862
        foo  1.070856  1.558910

## Normalización

#### Las tablas de frecuencia también se pueden normalizar para mostrar porcentajes en lugar de recuentos mediante el argumento de normalización

In [79]:
pd.crosstab(df["A"], df["B"], normalize=True)

B,A,B,C
A,,,
one,0.166667,0.166667,0.166667
three,0.083333,0.083333,0.083333
two,0.083333,0.083333,0.083333


In [80]:
pd.crosstab(df["A"], df["B"], normalize="columns")

B,A,B,C
A,,,
one,0.50,0.50,0.50
three,0.25,0.25,0.25
two,0.25,0.25,0.25


In [81]:
pd.crosstab(df["A"], df["B"], values=df["C"], aggfunc=np.sum)

B,A,B,C
A,,,
one,foobarfoobar,foobarfoobar,barfoobarfoo
three,barbar,foofoo,barbar
two,foofoo,barbar,foofoo


## Tiling - función cut()

#### La función cut() calcula agrupaciones para los valores de la matriz de entrada y, a menudo, se usa para transformar variables continuas en variables discretas o categóricas

In [82]:
ages = np.array([10, 15, 13, 12, 23, 25, 28, 59, 60])

In [83]:
pd.cut(ages, bins=3)

[(9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (26.667, 43.333], (43.333, 60.0], (43.333, 60.0]]
Categories (3, interval[float64, right]): [(9.95, 26.667] < (26.667, 43.333] < (43.333, 60.0]]

In [84]:
c = pd.cut(ages, bins=[0, 18, 35, 70])

In [85]:
c

[(0, 18], (0, 18], (0, 18], (0, 18], (18, 35], (18, 35], (18, 35], (35, 70], (35, 70]]
Categories (3, interval[int64, right]): [(0, 18] < (18, 35] < (35, 70]]

## Indicador de cálculo / variables ficticias - Pasar variables categoricas a 0 y 1 get_dummies()

#### Para convertir una variable categórica en un DataFrame "ficticio" o "indicador", por ejemplo, una columna en un DataFrame (una Serie) que tiene k valores distintos, puede derivar un DataFrame que contiene k columnas de 1 y 0 usando get_dummies()

In [86]:
df = pd.DataFrame({"key": list("bbacab"), "data1": range(6)})

In [87]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [88]:
pd.get_dummies(df["key"])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [89]:
# En algunas ocasiones, es recomendable utilizar un prefijo
dummies = pd.get_dummies(df["key"], prefix="key")

In [90]:
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [91]:
df[["data1"]].join(dummies)

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [93]:
# Ejemplo con Data Frame
df = pd.DataFrame({"A": ["a", "b", "a"], "B": ["c", "c", "b"], "C": [1, 2, 3]})

In [94]:
df

,A,B,C
0,a,c,1
1,b,c,2
2,a,b,3


In [95]:
pd.get_dummies(df)

,C,A_a,A_b,B_b,B_c
0,1,1,0,0,1
1,2,0,1,0,1
2,3,1,0,1,0


In [96]:
df = pd.DataFrame({"A": list("aaaaa"), "B": list("ababc")})

In [97]:
pd.get_dummies(df)

,A_a,B_a,B_b,B_c
0,1,1,0,0
1,1,0,1,0
2,1,1,0,0
3,1,0,1,0
4,1,0,0,1


In [98]:
pd.get_dummies(df, drop_first=True)

,B_b,B_c
0,0,0
1,1,0
2,0,0
3,1,0
4,0,1


## Ejemplos

In [99]:
np.random.seed([3, 1415])

In [100]:
n = 20

In [101]:
cols = np.array(["key", "row", "item", "col"])

In [102]:
df = cols + pd.DataFrame(
    (np.random.randint(5, size=(n, 4)) // [2, 1, 2, 1]).astype(str)
)

In [103]:
df.columns = cols

In [104]:
df = df.join(pd.DataFrame(np.random.rand(n, 2).round(2)).add_prefix("val"))

In [106]:
df.head(10)

,key,row,item,col,val0,val1
0,key0,row3,item1,col3,0.81,0.04
1,key1,row2,item1,col2,0.44,0.07
2,key1,row0,item1,col0,0.77,0.01
3,key0,row4,item0,col2,0.15,0.59
4,key1,row0,item2,col1,0.81,0.64
5,key1,row2,item2,col4,0.13,0.88
6,key2,row4,item1,col3,0.88,0.39
7,key1,row4,item1,col1,0.10,0.07
8,key1,row0,item2,col4,0.65,0.02
9,key1,row2,item0,col2,0.35,0.61


### Pivotar con agregaciones individuales

In [107]:
# Vamos a crear una pivot table en base a la tabla anterior
df.pivot_table(values="val0", index="row", columns="col", aggfunc="mean")

col,col0,col1,col2,col3,col4
row,,,,,
row0,0.77,0.605,NaN,0.860,0.65
row2,0.13,NaN,0.395,0.500,0.25
row3,NaN,0.310,NaN,0.545,NaN
row4,NaN,0.100,0.395,0.760,0.24


In [108]:
# Podemos utilizar el argumento "fill_value", para reemplazar todos los valores NA
df.pivot_table(
    values="val0",
    index="row",
    columns="col",
    aggfunc="mean",
    fill_value=0,
)

col,col0,col1,col2,col3,col4
row,,,,,
row0,0.77,0.605,0.000,0.860,0.65
row2,0.13,0.000,0.395,0.500,0.25
row3,0.00,0.310,0.000,0.545,0.00
row4,0.00,0.100,0.395,0.760,0.24


In [109]:
df.pivot_table(
    values="val0",
    index="row",
    columns="col",
    aggfunc="sum",
    fill_value=0,
)

col,col0,col1,col2,col3,col4
row,,,,,
row0,0.77,1.21,0.00,0.86,0.65
row2,0.13,0.00,0.79,0.50,0.50
row3,0.00,0.31,0.00,1.09,0.00
row4,0.00,0.10,0.79,1.52,0.24


In [110]:
df.pivot_table(index="row", columns="col", fill_value=0, aggfunc="size")

col,col0,col1,col2,col3,col4
row,,,,,
row0,1,2,0,1,1
row2,1,0,2,1,2
row3,0,1,0,2,0
row4,0,1,2,2,1


## Pivotar con múltiples agregaciones

#### También podemos realizar múltiples agregaciones. Por ejemplo, para realizar una suma y una media, podemos pasar una lista al argumento aggfunc.

In [111]:
df.pivot_table(
    values="val0",
    index="row",
    columns="col",
    aggfunc=["mean", "sum"],
)

mean                              sum                        
col   col0   col1   col2   col3  col4  col0  col1  col2  col3  col4
row                                                                
row0  0.77  0.605    NaN  0.860  0.65  0.77  1.21   NaN  0.86  0.65
row2  0.13    NaN  0.395  0.500  0.25  0.13   NaN  0.79  0.50  0.50
row3   NaN  0.310    NaN  0.545   NaN   NaN  0.31   NaN  1.09   NaN
row4   NaN  0.100  0.395  0.760  0.24   NaN  0.10  0.79  1.52  0.24

In [112]:
df.pivot_table(
    values=["val0", "val1"],
    index="row",
    columns="col",
    aggfunc=["mean"],
)

mean                                                           
      val0                             val1                          
col   col0   col1   col2   col3  col4  col0   col1  col2   col3  col4
row                                                                  
row0  0.77  0.605    NaN  0.860  0.65  0.01  0.745   NaN  0.010  0.02
row2  0.13    NaN  0.395  0.500  0.25  0.45    NaN  0.34  0.440  0.79
row3   NaN  0.310    NaN  0.545   NaN   NaN  0.230   NaN  0.075   NaN
row4   NaN  0.100  0.395  0.760  0.24   NaN  0.070  0.42  0.300  0.46

In [113]:
df.pivot_table(
    values=["val0"],
    index="row",
    columns=["item", "col"],
    aggfunc=["mean"],
)

mean                                                                   
      val0                                                                   
item item0             item1                         item2                   
col   col2  col3  col4  col0  col1  col2  col3  col4  col0   col1  col3  col4
row                                                                          
row0   NaN   NaN   NaN  0.77   NaN   NaN   NaN   NaN   NaN  0.605  0.86  0.65
row2  0.35   NaN  0.37   NaN   NaN  0.44   NaN   NaN  0.13    NaN  0.50  0.13
row3   NaN   NaN   NaN   NaN  0.31   NaN  0.81   NaN   NaN    NaN  0.28   NaN
row4  0.15  0.64   NaN   NaN  0.10  0.64  0.88  0.24   NaN    NaN   NaN   NaN

## Pasar de una lista a una columna de datos - explode()

In [114]:
keys = ["panda1", "panda2", "panda3"]

In [115]:
values = [["eats", "shoots"], ["shoots", "leaves"], ["eats", "leaves"]]

In [116]:
df = pd.DataFrame({"keys": keys, "values": values})

In [117]:
df

,keys,values
0,panda1,"[eats, shoots]"
1,panda2,"[shoots, leaves]"
2,panda3,"[eats, leaves]"


In [118]:
# Para poder separar los valores en una columna, utilizaremos "explode()"
df["values"].explode()

0      eats
0    shoots
1    shoots
1    leaves
2      eats
2    leaves
Name: values, dtype: object

In [119]:
df.explode("values")

,keys,values
0,panda1,eats
0,panda1,shoots
1,panda2,shoots
1,panda2,leaves
2,panda3,eats
2,panda3,leaves


In [120]:
df = pd.DataFrame([{"var1": "a,b,c", "var2": 1}, {"var1": "d,e,f", "var2": 2}])

In [121]:
df

,var1,var2
0,"a,b,c",1
1,"d,e,f",2


In [122]:
df.assign(var1=df.var1.str.split(",")).explode("var1")

,var1,var2
0,a,1
0,b,1
0,c,1
1,d,2
1,e,2
1,f,2
